In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def region_of_interest(img, vertices):
    # Define a blank matrix that matches the image height/width.
    mask = np.zeros_like(img)
    
    #check if image is grayscale or color
    shape = img.shape
    if(len(shape) == 3):               #its a color image
        mask_color = (255,)*shape[-1]   #shape[-1] = no. at last index
    else:                              #otherwise its a gray image
        mask_color = 255
      
    # Fill the polygon with white
    cv2.fillPoly(mask, vertices, mask_color)
    
    # Returning the image only where mask pixels match
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [ ]:
def filter_color(img):
    yellow_min = np.array([50, 80, 80], np.uint8)
    yellow_max = np.array([60, 255, 255], np.uint8)
    yellow_mask = cv2.inRange(img, yellow_min, yellow_max)

    white_min = np.array([200, 0, 200], np.uint8)
    white_max = np.array([255, 0, 255], np.uint8)
    white_mask = cv2.inRange(img, white_min, white_max)
    
    yelWhtMerge = cv2.bitwise_or(yellow_mask, white_mask)
    img = cv2.bitwise_and(img, img, yelWhtMerge)
    return img

In [ ]:
def draw_lines(img, lines, color=[255, 255, 255], thickness=3):
    '''
    Draw lines over lane mark.
    '''
    # If there are no lines to draw, exit.
    if lines is None:
        return
    
    # Create a blank image that matches the original in size.
    line_img = np.zeros_like(img)
    
    # Loop over all lines and draw them on the blank image.
    for line in lines:
        line = np.array(line)
        line = line.astype(int)
        for x1, y1, x2, y2 in line:
            cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)
    
    # Merge the image with the lines onto the original.
    img = cv2.addWeighted(img, 0.8, line_img, 0.8, 0.0)
    
    # Return the modified image.
    return img

In [ ]:
def applySobel(img, orient = 'x', kernel=3, th=100, blackWhite=(0, 255)):
    #cvt to gray
    if len(img.shape) > 2:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #extract vertical or horizontal line
    if orient == 'x':
        absSobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 1, 0))
    else:
        absSobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 0, 1))
        
    #rescale to 8 bit
    scaled = np.uint8(255*absSobel/np.max(absSobel))
    
    #create a blank image and apply thresh
    op = np.zeros_like(scaled)
    op[scaled > th] = blackWhite[1] #white
    op[scaled < th] = blackWhite[0] #black
    
    return op

In [ ]:
cap = cv2.VideoCapture('../dataset/dayLane.webm')

fourcc = cv2.VideoWriter_fourcc(*'VP90')

#isColor = False , means videowriter expects a gray image
out = cv2.VideoWriter('./dataset/histSobelx.webm', fourcc, 25.0, (int(cap.get(3)), int(cap.get(4))), isColor=False)

#get height and width of frame
ret, frame = cap.read()
height, width, channel = frame.shape

#keeping the portion of image which contains lane
topLeftPt = (width*0.30, height*0.60)
topRightPt = (width*0.70, height*0.60)

#this portion contain road
region_of_interest_points = [
(0, height*0.90),
#(0, height*0.60),
topLeftPt,
topRightPt,
#(width, height*0.60),
(width, height*0.90),
]

i = 1  #image count
while(cap.isOpened()):
    # Take each frame
    ret, frame = cap.read()
    
    #resize frame
    # frame = cv2.resize(frame, (int(frame.shape[0]), int(frame.shape[1]*0.8)), interpolation = cv2.INTER_AREA)
    
    #cv2.imshow('original', frame)
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit =  2, tileGridSize = (8,8))
    img = clahe.apply(img)
    cv2.imshow('clahe', img)
    img = cv2.Canny(img, 80, 185)
    img = applySobel(img, 'x', 3)
    img = region_of_interest(img, np.array([region_of_interest_points], np.int32))
    #out.write(img)
    cv2.imshow('img', img)
    k = cv2.waitKey(25) & 0xFF  #get pressed key
    if k == ord('q'):
        break
    elif k == ord('s'):
        cv2.imwrite('dataset/ld'+str(i)+'.jpg', frame)
        i += 1

cap.release()
out.release()
cv2.destroyAllWindows()